In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

### Data

In [2]:
raw_csv_data = np.loadtxt("Audiobooks_data.csv", delimiter=",")
raw_csv_data

array([[8.7300e+02, 2.1600e+03, 2.1600e+03, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [6.1100e+02, 1.4040e+03, 2.8080e+03, ..., 0.0000e+00, 1.8200e+02,
        1.0000e+00],
       [7.0500e+02, 3.2400e+02, 3.2400e+02, ..., 1.0000e+00, 3.3400e+02,
        1.0000e+00],
       ...,
       [2.8671e+04, 1.0800e+03, 1.0800e+03, ..., 0.0000e+00, 2.9000e+01,
        0.0000e+00],
       [3.1134e+04, 2.1600e+03, 2.1600e+03, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [3.2832e+04, 1.6200e+03, 1.6200e+03, ..., 0.0000e+00, 9.0000e+01,
        0.0000e+00]])

In [3]:
# first column (id), last (target)
unscaled_input_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:, -1]

### Pre balancing shuffle of data

In [14]:
# Getting better predictos without pre balancing shuffle of data

# shuffled_unscaled_indices = np.arange(unscaled_input_all.shape[0])
# np.random.shuffle(shuffled_unscaled_indices)

# unscaled_input_all = unscaled_input_all[shuffled_unscaled_indices]
# targets_all = targets_all[shuffled_unscaled_indices]

### balancing

In [5]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
                indices_to_remove.append(i)

In [6]:
unscaled_inputs_equal_priors = np.delete(unscaled_input_all, indices_to_remove, axis=0)
targets_equals_priors = np.delete(targets_all, indices_to_remove, axis=0)

### standardize inputs

In [7]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

### Shuffle data

In [8]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equals_priors[shuffled_indices]

### split dataset into train, validation and test

In [9]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8 * samples_count)
validation_samples_count = int (0.1 * samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count + validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count + validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count + validation_samples_count:]
test_targets = shuffled_targets[train_samples_count + validation_samples_count:]

In [10]:
# total, count, total / count
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1784.0 3579 0.4984632578932663
221.0 447 0.49440715883668906
232.0 448 0.5178571428571429


### Save .npz files

In [11]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)